## Import statements

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from math import log
import ast
import statsmodels.stats.multitest as multi
import pingouin as pg

from matplotlib_venn import venn3
import holoviews as hv
from holoviews import opts, dim
import holoviews.plotting.mpl
import itertools as it

In [ ]:
from src.functions import read_raw_protein_file, extract_datamatrix, rename_experiment, extract_datamatrix_DIA
from src.functions import extract_datamatrix_copynumber, extract_Parameters, imputation_normal_distribution
from src.functions import calculate_quartile_numbers, intersection, return_kegg_coverage, calculate_pathway_percentage

## Dataset pre-processing

In [ ]:
## ID mapping
ID_perseus = pd.read_csv('annotations/Perseus_annotation_file.csv')
IDmapping_Perseus_UniprotAC_to_Genenames = dict(zip(ID_perseus['Protein ID'], ID_perseus['Gene names']))
IDmapping_Perseus_UniprotAC_to_Proteinnames = dict(zip(ID_perseus['Protein ID'], ID_perseus['Protein names']))
IDmapping_Perseus_Genenames_to_Proteinnames = dict(zip(ID_perseus['Gene names'], ID_perseus['Protein names']))
IDmapping_Perseus_UniprotAC_to_Genename_UniID = dict(zip(ID_perseus['Protein ID'], ID_perseus['Genename_ProteinID']))

## tissue specificity RNA HPA
tissue_specificity = pd.read_csv('annotations/tissue_specificity_RNA_HPA.csv')
mapping_tissuespecificity_tissue = dict(zip(tissue_specificity['Gene'], tissue_specificity['RNA TS TPM']))

labelfile = pd.read_csv('annotations/Experiment annotation file.csv')
labelfile_pcc = pd.read_csv('datasets/PCC/Annotation_file_PCC.csv')

In [ ]:
## Datasets
ProteinGroups = 'datasets/Atlas/ProteinGroups_minratio2.txt'
ProteinGroups_copynumber = 'datasets/Atlas/proteinGroups_ratio2_10528_copynumber.txt'
ProteinGroups_pcc = 'datasets/PCC/Proteins_20200803_094937_HLAtlas_PrimaryCellCulture_v14_Report.csv'
Copynumber_overview = 'datasets/Atlas/proteinGroups_ratio2_10528_copynumber_overview.txt'
columns_reorder = pd.read_csv('annotations/column_reorder.csv')['reorder'].to_list()
celltypes = labelfile.loc[labelfile['Label5']=='Primary_cell_types']['Label4'].to_list()
grouping1 = dict(zip(labelfile['Label4'], labelfile['Label6']))

#### LFQ intensity dataset pre-processing

In [ ]:
data_lfq = extract_datamatrix(ProteinGroups)
data_lfq = data_lfq.rename(mapper = dict(zip(labelfile['Name'], labelfile['Label4'])), axis = 1)
data_lfq = data_lfq[columns_reorder].dropna(how = 'all')
data_lfq.index.name = 'Protein ID'
data_lfq_log10 = np.log10(data_lfq)
data_lfq_log10_impute = imputation_normal_distribution(data_lfq_log10)

In [ ]:
## LFQ intensity dataset getting median values
col_to_keep = ['Liver', 'HepA', 'PorV', 'hHEP', 'hLSEC', 'hKC', 'hHSC']
data_lfq_median = data_lfq.groupby(grouping1, axis = 1).median()[col_to_keep]
data_lfq_log10_median = data_lfq_log10.groupby(grouping1, axis = 1).median()[col_to_keep]

#### Protein copy number dataset pre-processing

In [ ]:
data_cpno = extract_datamatrix_copynumber(ProteinGroups_copynumber)
data_cpno = data_cpno.iloc[:, 0:34].astype(float)
data_cpno = data_cpno.rename(mapper = dict(zip(labelfile['Name_copynumber'], labelfile['Label4'])), axis = 1)
data_cpno = data_cpno[columns_reorder].dropna(how = 'all')
data_cpno.index.name = 'Protein ID'
data_cpno_log10 = np.log10(data_cpno)
data_cpno_median = data_cpno.groupby(grouping1, axis = 1).median()[col_to_keep]
data_cpno_log10_median = data_cpno_log10.groupby(grouping1, axis = 1).median()[col_to_keep]

#### Primary cell culture experiment dataset pre-processing

In [ ]:
Report_pcc = pd.read_csv(ProteinGroups_pcc)
experimental_columns = labelfile_pcc['Sample ID'].tolist()
col_tokeep = ['Protein ID', 'Gene names', 'PG.ProteinDescriptions'] + experimental_columns
data_pcc_raw = extract_datamatrix_DIA(Report_pcc, labelfile=labelfile_pcc)[col_tokeep]
IDmapping_pcc_UniprotID_to_Genename = dict(zip(data_pcc_raw['Protein ID'], data_pcc_raw['Gene names']))
IDmapping_pcc_UniprotID_to_ProteinName = dict(zip(data_pcc_raw['Protein ID'], data_pcc_raw['PG.ProteinDescriptions']))
data_pcc_raw = data_pcc_raw.set_index('Protein ID').drop(['Gene names','PG.ProteinDescriptions'] , axis = 1).astype(np.number)
data_pcc_raw.columns = pd.MultiIndex.from_tuples(zip(labelfile_pcc['Label3'], labelfile_pcc['Sample ID']))
data_pcc_filtered = data_pcc_raw.dropna(how='all')
data_pcc_log2 = np.log2(data_pcc_filtered)
data_pcc_log2_impute = imputation_normal_distribution(data_pcc_log2)

In [ ]:
data_pcc_raw_supp = data_pcc_raw.copy()
data_pcc_raw_supp['Gene name'] = data_pcc_raw_supp.index.map(IDmapping_pcc_UniprotID_to_Genename)
data_pcc_raw_supp['Protein name'] = data_pcc_raw_supp.index.map(IDmapping_pcc_UniprotID_to_ProteinName)

#### Importing molecular weight data

In [ ]:
MW = pd.read_csv(ProteinGroups_copynumber, sep ='\t', low_memory=False)[['Majority protein IDs', 'Mol. weight [kDa]']][1:]
MW['Mol. weight [kDa]'] = MW['Mol. weight [kDa]'].astype(float)
MW['Protein ID']= MW['Majority protein IDs'].str.split(';').str[0];

#### Patient sample dataset pre-processing

In [ ]:
annotation_patients = pd.read_csv('datasets/Patients/PatientSamples_annotation_file.csv')
data_patients = pd.read_csv('datasets/Patients/PatientSamples_proteome.csv')
data_patients['Protein ID']=data_patients['Protein IDs'].str.split(';').str[0]
data_patients_processed = data_patients.drop(['Protein IDs', 'Gene names'], axis=1)
IDmapping_patients_UniprotID_to_GeneName=dict(zip(data_patients_processed['Protein ID'], data_patients_processed['Gene name']))

In [ ]:
data_patients_long = data_patients_processed.melt(id_vars=['Gene name', 'Protein ID'], var_name='Sample ID', value_name='Intensity [Log2]')
data_patients_long['grouping']=data_patients_long['Sample ID'].map(dict(zip(annotation_patients['Sample ID'], annotation_patients['Grouping'])))
for variate in ['BMI', 'age', 'gender']:
    dict_variate = dict(zip(annotation_patients['Sample ID'], annotation_patients[variate]))
    data_patients_long[variate]=data_patients_long['Sample ID'].map(dict_variate)
data_patients_long['obesity']=np.where(data_patients_long['BMI']>30, 1, 0)

## Figure 1

#### Depth 

In [ ]:
depth_pg = data_lfq.groupby(dict(zip(labelfile['Label4'], labelfile['Label5'])), axis = 1).median().count()
depth_pg['total'] = data_lfq.shape[0]
fig, ax = plt.subplots(figsize = (4,4))
plt.bar(x = depth_pg.index, height = depth_pg, width = 0.4, edgecolor= 'black', facecolor = 'steelblue')
for i in np.arange(4):
    plt.annotate(depth_pg.iloc[i], [depth_pg.index[i], depth_pg.iloc[i]])
plt.ylabel('Protein groups')
plt.xticks(rotation = 20)
plt.title('Proteome depth');

#### Abundance

In [ ]:
fig, ax = plt.subplots(figsize = (4, 4))
df = data_lfq_log10_median
df['Gene name']=df.index.map(IDmapping_Perseus_UniprotAC_to_Genenames)
top_50_dict = {}
for i in ['Liver', 'HepA', 'PorV', 'hHEP', 'hLSEC', 'hKC', 'hHSC']:
    top_50=df.sort_values(by=i, ascending=False)['Gene name'][:50].tolist()
    top_50_dict[i]=top_50
    y = sorted(df[[i]].dropna().reset_index()[i])[::-1]
    plt.plot(y)
    plt.legend(labels = df.columns)
plt.ylabel('LFQ intensity [Log10]')
plt.xlabel('Abundance rank')
plt.title('Abundance rank');

#### PCA of all samples 

In [ ]:
df_PCA = data_lfq_log10_impute.rename(mapper = dict(zip(labelfile['Label4'], labelfile['Label3'])), axis = 1)
df_PCA = df_PCA.rename_axis('Sample type', axis = 1)
df = df_PCA.T.reset_index()
features = df.columns[1:]

In [ ]:
x=df.loc[:, features].values
y=df.loc[:, 'Sample type'].values
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])
finalDf = pd.concat([principalDf, df['Sample type']], axis = 1)
PC1= pca.explained_variance_ratio_[0]
PC2= pca.explained_variance_ratio_[1]
PC1 = "{:.1%}".format(PC1)
PC2 = "{:.1%}".format(PC2)

In [ ]:
fig, ax = plt.subplots(figsize = (4,4))
ax.set_xlabel('Principal Component 1 ('+ str(PC1) + ')')
ax.set_ylabel('Principal Component 2 ('+ str(PC2) + ')')
ax.set_title('Principal component analysis')

targets = df['Sample type'].unique()
for target in targets:
    indicesToKeep = finalDf['Sample type'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , cmap =  'set3'
               , s = 80, edgecolors='black', alpha=0.7)
ax.legend(targets, bbox_to_anchor = [1.05, 0.8], framealpha=0.1)
ax.grid()

#### Kegg pathway coverage

In [ ]:
## Protein coding gene and keggID annotated from Perseus
df_pcg = pd.read_csv('annotations/Kegg/gene list_Perseus_keggID_18836.csv')[1:]
## kegg pathway ID and geneID downloaded from Kegg
kegg_pathway_gene = pd.read_csv('annotations/Kegg/kegg_pathway_genes.csv')
## kegg pathway ID and pathway name downloaded from Kegg
pathwayID= pd.read_csv('annotations/Kegg/keggPathwayID.csv', names=['pathwayID', 'pathwayName'])
IDmapping_keggID_to_Genesymbol= dict(zip(df_pcg['keggID'], df_pcg['Gene name']))
## protein coding genes keggID
proteincodinggenes = df_pcg['Gene name']

In [ ]:
pathways= pd.merge(kegg_pathway_gene, pathwayID, on='pathwayID')
pathways['Gene names']= pathways['geneID'].map(IDmapping_keggID_to_Genesymbol.get)
pathways = pathways.applymap(lambda x: str(x).replace(' - Homo sapiens (human)', ''))

pathway_dict={}
for i in range(pathways.shape[0]):
    current_key = pathways.iloc[i]['pathwayName']
    current_value = pathways.iloc[i]['Gene names']
    pathway_dict.setdefault(current_key, [])
    pathway_dict[current_key].append(current_value)
pathway_dict_pathways = list(pathway_dict.keys())
pathway_dict_genes = list(pathway_dict.values())

pcg_dict = {}
for i in range(len(pathway_dict)):
    pcg_dict[pathway_dict_pathways[i]] = list(set(pathway_dict_genes[i]).intersection(set(proteincodinggenes)))
df = data_lfq_log10_median.copy()
df['Gene name'] = df.index.map(IDmapping_Perseus_UniprotAC_to_Genenames)
df = df.set_index('Gene name')
kegg_coverage = return_kegg_coverage(df, pcg_dict, pathway_dict=pathway_dict).set_index('pathways', drop = True)
kegg_coverage = kegg_coverage.sort_values(by = 'percentage of covered genesall')

In [ ]:
df = kegg_coverage
fig, ax=plt.subplots(figsize=(4,4))
y=np.arange(df.shape[0])
width = df['percentage of covered genesall']
ax.grid(ls='dashed', color= 'black', alpha=0.5)
plt.barh(y, width, height = 1, edgecolor = 'lightblue')
plt.title('KEGG coverage')
plt.xlabel('Coverage rate')
plt.ylabel('Pathways');

## Figure 2

#### Venn between tissue types

In [ ]:
df = data_lfq_log10_median.copy()
tissue = ['HepA', 'PorV', 'Liver']
df_tissue = df[tissue].dropna(how = 'all')

#### Reviewer question: what is the nature of the proteins 'unique' to HepA and PorV?

In [ ]:
unique_in_vessels_proteinids = df_tissue[(df_tissue['Liver'].isnull()) & (df_tissue['HepA'].notnull()) & (df_tissue['PorV'].notnull())].index
unique_in_eithervessel_proteinids= df_tissue[df_tissue['Liver'].isnull()].index
unique_in_vessels_genenames = [IDmapping_Perseus_UniprotAC_to_Genenames[i] for i in unique_in_vessels_proteinids]
unique_in_vessles_proteinids_rank = df_tissue.sort_values(by='HepA', ascending = False).assign(rank_hepa = np.arange(df_tissue.shape[0])).loc[unique_in_vessels_proteinids]['rank_hepa']

In [ ]:
fig, ax = plt.subplots(figsize = (4, 4))
v = venn3(subsets = [set(df_tissue[i].dropna().index) for i in df_tissue.columns], set_labels=tissue, alpha = 0.3)
print(df_tissue.count())

#### Cumulative abundance tissue types

In [ ]:
data = data_lfq_median.copy()
df_new = pd.DataFrame(index = data.index)
for column in data.columns:
    sorted_column = data[[column]].sort_values(by = column, ascending = False)
    cum_column = np.cumsum(sorted_column)    
    df_new['cum_' + column] = cum_column/cum_column.max()

In [ ]:
data = df_new
fig, ax = plt.subplots(figsize = (10, 4), squeeze = False)
plt.subplots_adjust(wspace = 0.3)
j = 1
for i in ['cum_Liver', 'cum_HepA', 'cum_PorV']:
    x = np.arange(data.shape[0])
    y = data.sort_values(by = i)[i]
    x0 = x[:10]
    y0 = y[:10]
    plt.subplot(1, 3, j)
    plt.plot(x, y)
    plt.plot(x0, y0, 'ro')
    plt.title(i.strip('cum_'))
    plt.ylabel('Cumulative abundance [%]')
    plt.ylim(-0.05, 1.05)
    plt.yticks([0, 0.25, 0.5, 0.75, 1])
    plt.xticks(ticks=[0, 2500, 5000, 7500])
    j += 1
    annotations = calculate_quartile_numbers(data, i)
    top10=data.sort_values(by=i).index[:10]
    top10_gene = [IDmapping_Perseus_UniprotAC_to_Genenames[a] for a in top10][::-1]
    for i in np.arange(10):
        plt.text(x=1000, y=0.05+0.05*i, s= top10_gene[i])
    for b in np.arange(1, 5):
        plt.annotate(xy=(5200, 0.05*b), s= 'Q'+str(b)+':'+str(annotations[b]))
plt.savefig('figures/Cumulative_tissuetypes.pdf')

#### Principal component analysis of tissue types

In [ ]:
tissue_cls = ['HepA', 'PorV', 'Liver', 'Cell_lines']
df_PCA_t = df_PCA[tissue_cls]
df = df_PCA_t.T.reset_index()
features = df.columns[1:]

In [ ]:
x=df.loc[:, features].values
y=df.loc[:, 'Sample type'].values
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])
finalDf = pd.concat([principalDf, df['Sample type']], axis = 1)
PC1= pca.explained_variance_ratio_[0]
PC2= pca.explained_variance_ratio_[1]
PC1 = "{:.1%}".format(PC1)
PC2 = "{:.1%}".format(PC2)

In [ ]:
d = pca.components_.T * np.sqrt(pca.explained_variance_)

df_features=pd.DataFrame(d)
df_features.columns=['loading_PC1', 'loading_PC2']
ID=list(df.T.index[1:])
df_features['Protein IDs']=ID
df_features['Gene names']=df_features['Protein IDs'].map(IDmapping_Perseus_UniprotAC_to_Genenames.get)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (8, 4))
plt.subplots_adjust(wspace = 0.3)
ax1.set_xlabel('Principal Component 1 ('+ str(PC1) + ')')
ax1.set_ylabel('Principal Component 2 ('+ str(PC2) + ')')
ax1.set_title('Principal component analysis')
targets = df['Sample type'].unique()
for target in targets:
    indicesToKeep = finalDf['Sample type'] == target
    ax1.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , cmap =  'set3'
               , s = 80, edgecolors='black')
ax1.legend(targets, framealpha=0.1)

loading = pca.components_.T * np.sqrt(pca.explained_variance_)
ax2.scatter(loading[:, 0], loading[:, 1], edgecolors = 'white', color = 'steelblue')
ax2.set_xlabel('PC1')
ax2.set_ylabel('PC2');

#### Reviewer question: where are blood proteins on the loading plot, do they contribute to the seperation between cell lines and liver tissues?

In [ ]:
blood_proteins = ['ALB', 'APOA1', 'HBA1', 'HBB', 'A2M', 
                  'APOA1', 'APOA4','APOA2',
                 'IGHG4', 'IGHG1', 'IGHG2', 'IGHG3','IGHA1', 'IGHM',
                 'A1AT', 'HP']

In [ ]:
df_test=df_features.copy()
df_test['blood protein']=np.where(df_test['Gene names'].isin(blood_proteins), 1, 0)

In [ ]:
fig, ax=plt.subplots(figsize=(4,4))
sns.scatterplot(x='loading_PC1', y='loading_PC2', hue='blood protein', data=df_test,
               legend=False)
sns.scatterplot(x='loading_PC1', y='loading_PC2', hue='blood protein', data=df_test[df_test['blood protein']==1],
                palette={0:'white', 1:'red'}, 
               legend=False)
plt.savefig('figures/PCA_tissue_bloodproteins.pdf', dpi=120, bbox_inches='tight')

#### Reviewer question: what are abundance ranks of the driver proteins in figure 2d?

In [ ]:
blue_proteins = ['MYH11', 'DES', 'TPM', 'CNN1', 'LMOD1', 'COL21A1', 'FBLN', 'OGN', 'ELN', 'PRELP']
red_proteins = ['MBL2', 'CYP4A11', 'CYP2E1', 'CYP2D6','CYP4F3', 'ALDH8A1', 'RDH16', 'UGT1A4', 'BHMT2']
black_proteins = ['HIST2H3A', 'IGF2BP2', 'IGF2BP1', 'IGFBP3', 'TRIP13', 'HMGA2', 'CYR61' ]

In [ ]:
a={i:'blue_vessels' for i in blue_proteins}
a.update({i:'red_liver' for i in red_proteins})
a.update({i:'black_celllines' for i in black_proteins})

In [ ]:
df_tissues=pd.concat([df_PCA_t[a].median(axis=1) for a in [['HepA', 'PorV'], ['Cell_lines'], ['Liver']]], axis=1)
df_tissues.columns=['vessels', 'cell lines', 'liver']
for col in df_tissues.columns:
    df_tissues=df_tissues.sort_values(by=col, ascending=False)
    df_tissues=df_tissues.assign(new_col=np.arange(df_tissues.shape[0]))
    df_tissues=df_tissues.rename({'new_col':'rank_{}'.format(col)}, axis=1)
df_tissues['Gene name']=df_tissues.index.map(IDmapping_Perseus_UniprotAC_to_Genenames)
df_tissues['color']=df_tissues['Gene name'].map(a)
df_tissues_colored=df_tissues[df_tissues['color'].notnull()].sort_values(by=['color'], ascending=False)

In [ ]:
df_tissues_colored.to_csv('results/tissue_pca_driving_proteins.csv')

## Figure 3

#### Venn between cell types

In [ ]:
df = data_lfq_log10_median.copy()
celltypes_grouped = ['hHEP', 'hHSC', 'hLSEC', 'hKC']
df_cell = df[celltypes_grouped].dropna(how = 'all')

In [ ]:
import src.venn as venn
labels = venn.get_labels([df_cell[[i]].dropna().index for i in celltypes_grouped])
names=['hKC', 'hHSC', 'hHEP', 'hLSEC']
fig, ax = venn.venn4(labels, names=names, figsize= (3,3), fontsize = 8)

#### Proteome similarity

In [ ]:
data = data_lfq_log10_median.copy()
cell_types = ['hHSC', 'hLSEC', 'hKC', 'hHEP']
data = data[cell_types]

In [ ]:
df=data_lfq_log10.copy()
df['Gene name']=df.index.map(IDmapping_Perseus_UniprotAC_to_Genenames)

In [ ]:
data_lfq_log10_median['Gene name']=data_lfq_log10_median.index.map(IDmapping_Perseus_UniprotAC_to_Genenames)

In [ ]:
df2 = data_lfq_log10_median.copy()
df2['Gene name'] = df2.index.map(IDmapping_Perseus_UniprotAC_to_Genenames)
df2.sort_values(by='HepA', ascending = False, inplace= True)
df2['rank'] = np.arange(df2.shape[0])

In [ ]:
corr = data.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
e = np.array(data.corr())
c = sns.clustermap(data = corr, mask = mask, figsize = (4,4), cmap = 'RdBu', vmin = 0, vmax = 1, linecolor = 'white', linewidth = 1.5, annot = e, square=True)
plt.savefig('figures/figure3b.pdf', dpi=120, bbox_inches='tight')

#### Cumulative abundance cell types

In [ ]:
data = data_lfq_median.copy()
df_new = pd.DataFrame(index = data.index)
for column in data.columns:
    sorted_column = data[[column]].sort_values(by = column, ascending = False)
    cum_column = np.cumsum(sorted_column)    
    df_new['cum_' + column] = cum_column/cum_column.max()

In [ ]:
data = df_new
fig, ax = plt.subplots(figsize = (12, 4), squeeze = False)
plt.subplots_adjust(wspace = 0.35)
j = 1
for i in ['cum_hHEP', 'cum_hLSEC', 'cum_hKC', 'cum_hHSC']:
    x = np.arange(data.shape[0])
    y = data.sort_values(by = i)[i]
    x0 = x[:10]
    y0 = y[:10]
    plt.subplot(1, 4, j)
    plt.plot(x, y)
    plt.plot(x0, y0, "ro")
    plt.title(i.strip('cum_'))
    plt.ylim(-0.05, 1.05)
    plt.yticks([0, 0.25, 0.5, 0.75, 1])
    plt.ylabel('Cumulative abundance [%]')
    j += 1
    annotations = calculate_quartile_numbers(data, i)
    top10=data.sort_values(by=i).index[:10]
    top10_gene = [IDmapping_Perseus_UniprotAC_to_Genenames[a] for a in top10][::-1]
    for i in np.arange(10):
        plt.text(x=1000, y=0.05+0.05*i, s= top10_gene[i])
    for b in np.arange(1, 5):
        plt.annotate(xy=(5200, 0.05*b), s= 'Q'+str(b)+':'+str(annotations[b]))
plt.savefig("figures/Cumulative_celltypes.pdf")

#### Subcellular mass composition

In [ ]:
sub_loc = pd.read_csv('annotations/HPA/subcellular_location.csv')
sub_loc = sub_loc[sub_loc['Reliability']!='Uncertain']
new_df = pd.DataFrame(sub_loc['GO id'].str.split(';').to_list(), index = sub_loc['Gene name']).stack()
new_df = new_df.reset_index([0, 'Gene name'])
new_df.columns = ['Gene name', 'GO id']
new_df['Gene name'] = new_df['Gene name'].astype(str)
sub_loc = new_df.copy()
sub_loc = sub_loc.groupby(['GO id'])['Gene name'].apply(list).reset_index()
pathway_location = dict(zip(sub_loc['GO id'], sub_loc['Gene name']))

In [ ]:
data = data_lfq_median.copy()
data['Gene name'] = data.index.map(IDmapping_Perseus_UniprotAC_to_Genenames)
data = data.set_index('Gene name')
unassigned = list(set(data.index) - set(new_df['Gene name']))
pathway_location['unassigned'] = unassigned
per_loc = calculate_pathway_percentage(pathway_dictionary=pathway_location, data = data).T
#per_loc = per_loc.div(per_loc.sum(axis=1), axis=0).T

In [ ]:
remaining = per_loc.sort_values(by = 'hHEP')[:16].index
sum_remaining = per_loc.loc[remaining].sum()
per_loc2 = per_loc.drop(labels = remaining)
df = per_loc2.copy().T
df['remaining'] = sum_remaining
df = df.T.sort_values(by = 'hHEP', ascending = False)
order = list(df.index[1:]) + ['unassigned']
df = df.loc[order]
df_nor = df.div(df.sum(axis=0), axis=1)
#df.to_csv('datasets/subcellular mass composition before normalization.csv')
#df_nor.to_csv('datasets/subcellular mass composition after normalization.csv')

In [ ]:
x = df_nor.T
ax=x.plot(kind='barh', stacked=True, figsize=(6,4), title='Subcellular mass composition', legend= False)
ax.legend(bbox_to_anchor = [1, 1]);

#### Subcellular co-localization 

In [ ]:
hv.extension('bokeh')
%output size=100

In [ ]:
GO=pd.read_csv('annotations/GO_colocalization.csv')['GO terms'].tolist()
location_table=pd.DataFrame(list(it.combinations(GO, 2)))
location_table.columns=['source', 'target']

In [ ]:
data = data_lfq_median.copy()
data['Gene name'] = data.index.map(IDmapping_Perseus_UniprotAC_to_Genenames)
data = data.set_index('Gene name')
hep_GNs = list(data[['hHEP']].dropna().index.unique())

overlap=[]
overlap_gene= []
for index, row in location_table.iterrows():
    source_GO=location_table.source.loc[index]
    target_GO=location_table.target.loc[index]
    genes_source = pathway_location[source_GO]
    genes_target =pathway_location[target_GO]
    overlap_genes = list(set(genes_source)&set(genes_target)&set(hep_GNs))
    overlap.append(len(overlap_genes))
    overlap_gene.append(overlap_genes)

location_table['value'] = overlap
location_table['overlapping_genes'] = overlap_gene

In [ ]:
links=location_table[['source', 'target', 'value']]
location_table.to_csv('datasets/colocalization.csv')
chords = hv.Chord(links)
chords

## Figure S2

#### Protein class composition

In [ ]:
annotations_class = pd.read_csv('annotations/HPA/proteinClass.csv', usecols = ['protein class', 'protein list'])
for i in np.arange(annotations_class.shape[0]):
    content = annotations_class['protein list'].loc[i]
    annotations_class['protein list'].loc[i] = ast.literal_eval(content)
df = annotations_class
dict_proteinClass = dict(zip(df['protein class'], df['protein list']))

In [ ]:
data = data_lfq.copy()
data['Gene name'] = data.index.map(IDmapping_Perseus_UniprotAC_to_Genenames)
data = data.set_index('Gene name')
per_proteinClass = calculate_pathway_percentage(dict_proteinClass, data)

In [ ]:
df = per_proteinClass.reset_index()
df_anova = df.melt(id_vars = 'Samples').set_index('Pathways')
df_anova['grouping1'] = df_anova['Samples'].map(grouping1)

In [ ]:
per_proteinClass_celltype= per_proteinClass.T.groupby(by = grouping1, axis = 1).median().round(3)[['hHEP', 'hKC', 'hLSEC', 'hHSC', 'Liver']]
df = per_proteinClass_celltype.sort_values(by='Liver', ascending =False)
df = df[['Liver', 'hHEP', 'hKC', 'hLSEC', 'hHSC']]
annotation = df.copy()*100
A = sns.clustermap(data = df, annot = np.array(annotation), linewidth = 0.1, row_cluster = False, col_cluster = False, figsize = (4,8), cmap = 'Blues', vmin = 0)
plt.savefig('figures/S1.pdf', dpi=120, bbox_inches='tight')

#### Mirochondria mass composition

In [ ]:
resp_units = pd.read_csv('annotations/respiratory_subunits_core.csv')
class_enzymes = pd.read_csv('annotations/HPA/Protein class/protein_class_Enzymes.csv', usecols = ['Gene', 'Reliability (IH)'])
df = class_enzymes
enzymes = df[df['Reliability (IH)'] != 'Uncertain']['Gene']
Mitochondria = pathway_location['Mitochondria (GO:0005739)']
MT_enzymes = list(set(Mitochondria) & set(enzymes))

In [ ]:
total_SLCs=[i for i in list(IDmapping_Perseus_UniprotAC_to_Genenames.values()) if i.startswith('SLC')]
SLC25=pd.read_csv('annotations/Mitochondrial_solute_carriers.csv')['Genename'].tolist()

In [ ]:
MT_ribosome = ['MRPL1','MRPL10','MRPL11','MRPL12','MRPL14','MRPL19','MRPL2','MRPL20','MRPL21','MRPL22','MRPL23','MRPL28',
               'MRPL36','MRPL37','MRPL38','MRPL40','MRPL41','MRPL42','MRPL43','MRPL44','MRPL45','MRPL46','MRPL47','MRPL51',
               'MRPL52','MRPL54','MRPL57','MRPL58','MRPL9','MRPS10','MRPS11','MRPS14','MRPS15','MRPS16','MRPS18B','MRPS2',
               'MRPS21','MRPS22','MRPS23','MRPS25','MRPS26','MRPS28','MRPS31','MRPS34','MRPS35','MRPS36','MRPS5','MRPS7',
               'MRPS9']
respiratory_chain = resp_units[resp_units['Complex']!='ATPase']['Gene name'].tolist()
MTclass_dict = {'MT ribosomes' : MT_ribosome, 'SLC25': SLC25, 'MT enzymes' : MT_enzymes, 'Respiratory chain complex I-IV': respiratory_chain}
for i in resp_units['Complex'].unique():
    MTclass_dict[i] = list(resp_units[resp_units['Complex'] == i]['Gene name'])

In [ ]:
data = data_lfq_median.copy()
data['Gene name'] = data.index.map(IDmapping_Perseus_UniprotAC_to_Genenames)
data_mito = data[data['Gene name'].isin(Mitochondria)].set_index('Gene name')
per_mito = calculate_pathway_percentage(pathway_dictionary=MTclass_dict, data = data_mito).T.reset_index()
per_mito = per_mito.melt(id_vars = 'Pathways')

In [ ]:
data[data['Gene name'].isin(SLC25)].sum()[:-1] / data[data['Gene name'].isin(Mitochondria)].sum()[:-1] *100

In [ ]:
df = per_mito
fig, ax = plt.subplots(figsize = (4,4))
plt.scatter(x = df['Samples'], y = df['Pathways'], s = df['value']*400, c = 'lightblue', edgecolor = 'black');

In [ ]:
df_wide = df.pivot(index='Pathways', columns= 'Samples', values='value').round(3)
df_wide = df_wide[['Liver', 'hHEP', 'hKC', 'hLSEC', 'hHSC']]
df_wide=df_wide.sort_values(by='Liver', ascending=False)

In [ ]:
df_wide.to_csv('results/mitochondrial_mass_composition.csv')

In [ ]:
annotation = np.array(df_wide.round(3))*100
sns.clustermap(data = df_wide, annot = np.array(annotation), linewidth = 0.1, row_cluster = False, col_cluster = False, figsize = (4,8), cmap = 'Blues', vmin = 0)
plt.savefig('figures/MT_mass.pdf', dpi=120, bbox_inches='tight')

## Figure 4

#### Cell type uniquely detected proteins

In [ ]:
data = data_lfq_log10.copy()[celltypes]
vv = data.groupby(grouping1, axis =1).count()
markers = []
candidates= []
for i in np.arange(vv.shape[0]):
    item = vv.index[i]
    if vv.loc[item].sum() == 3 & vv.loc[item].max() == 3:
        markers.append(item)
        candidates.append(item)
    elif vv.loc[item].sum() ==2 & vv.loc[item].max() == 2:
        candidates.append(item)

In [ ]:
# Fisher exact test significant proteins (proteins that were uniquely detected in one cell type in at least 2/3 biological replicates)
df_fisher_sig = vv.loc[candidates]
for i in df_fisher_sig.columns:
    print('Fisher exact sig proteins for {} is {}'.format(i, df_fisher_sig[df_fisher_sig[i]!=0].shape[0]))

In [ ]:
df_m = data.loc[markers].groupby(grouping1, axis = 1).mean()
top10 = []
for col in df_m.columns:
    top = df_m.nlargest(10, col).index
    top10.extend(top)
df_top = data.loc[top10]
df_top = df_top.fillna(5)
c=sns.clustermap(df_top, cmap= 'bwr', xticklabels=False, yticklabels=False,
                col_cluster=False, row_cluster=False, linewidths=.6, figsize=(6,6), 
                 cbar_kws = dict(use_gridspec=False));
plt.savefig('figures/celltype_unique.pdf', dpi=120, bbox_inches='tight')

#### Protein copy number of cell type unique proteins

In [ ]:
cell_types = labelfile[labelfile['Label3'].isin(['hHEP', 'hKC', 'hLSEC', 'hHSC'])]['Label4'].tolist()
df1=data_cpno.copy().loc[markers][cell_types]
df2=data_lfq.copy().loc[markers][cell_types]
df1.columns=['LFQ intensity ' + i for i in df1.columns]
df2.columns=['Protein copy number ' + i for i in df2.columns]
df_new=df1.join(df2).sort_values(by =['LFQ intensity hHEP_D1', 'LFQ intensity hHSC_D1', 'LFQ intensity hKC_D1', 'LFQ intensity hLSEC_D1'])
df_new['Gene name']=df_new.index.map(IDmapping_Perseus_UniprotAC_to_Genenames)
df_new['Protein name']=df_new.index.map(IDmapping_Perseus_UniprotAC_to_Proteinnames)
df_new.to_csv('results/cell type unique proteins.csv')

#### Respiratory chain ratio between cell types modelling 

In [ ]:
data = data_cpno_median.copy()
data['Gene name'] = data.index.map(IDmapping_Perseus_UniprotAC_to_Genenames)
data_resp = data[data['Gene name'].isin(resp_units['Gene name'])]

In [ ]:
data_resp_annotated = pd.merge(left=data_resp.reset_index(), right=resp_units[['Gene name', 'Complex', 'Protein name']], on='Gene name').set_index('Protein ID')

In [ ]:
data_resp_annotated.to_csv('results/copynumber_resp_all.csv')

In [ ]:
data_all = data_cpno.copy()
data_all['Gene name'] = data_all.index.map(IDmapping_Perseus_UniprotAC_to_Genenames)
data_all_OP = data_all[data_all['Gene name'].isin(resp_units['Gene name'])].drop('Gene name', axis=1)

In [ ]:
df_new = pd.DataFrame(data_all_OP.sum(), columns=['summed'])
df_new['group'] = df_new.index.map(dict(zip(labelfile['Label4'], labelfile['Label3'])))
df_new['summed'] = np.log10(df_new['summed'])
df_new=df_new[df_new['group'].isin(['Liver', 'hHEP', 'hKC', 'hLSEC', 'hHSC'])]

average = df_new.groupby('group').mean()
sd = df_new.groupby('group').std()

In [ ]:
fig, ax=plt.subplots(figsize=(4,4))
plt.bar(x=range(average.shape[0]), height=average['summed'], yerr=sd['summed'], width=0.6)
plt.xticks(ticks=np.arange(average.shape[0]), labels=average.index)
plt.savefig('figures/copynumber_fig.pdf')

In [ ]:
df=data_resp
df_new=df.dropna(subset=['hKC', 'hHEP'])
x, y = df_new['hKC'], df_new['hHEP']
x, y = np.log10(x), np.log10(y)

In [ ]:
IDmapping_GeneName_to_OPComplex = dict(zip(resp_units['Gene name'], resp_units['Complex']))
df_check = df.copy()
df_check['Complex'] = df_check['Gene name'].map(IDmapping_GeneName_to_OPComplex)

In [ ]:
df_atpase= df_check[df_check['Complex']=='ATPase'].sort_values(by='hHEP')
df_complex4= df_check[df_check['Complex']=='Complex4'].sort_values(by='hHEP')
df_complex1=df_check[df_check['Complex']=='Complex1'].sort_values(by='hHEP')
df_complex2=df_check[df_check['Complex']=='Complex2'].sort_values(by='hHEP')
df_complex3=df_check[df_check['Complex']=='Complex3'].sort_values(by='hHEP')

In [ ]:
OP_sum = df_check.drop('Gene name', axis=1).groupby('Complex').sum()

In [ ]:
OP_sum.to_csv('results/oxidative_phosphorylation_copynumbers.csv')

In [ ]:
df = data_resp
fig, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize = (14.5, 4))
plt.subplots_adjust(wspace = 0.35)
cols = ['hKC', 'hHSC', 'hLSEC']
i = 0
for ax in [ax0, ax1, ax2]:
    col = cols[i]
    df_new = df.copy()
    df_new = df_new.dropna(subset = [col, 'hHEP'])
    x, y = df_new[col], df_new['hHEP']
    x1, y1 = np.log10(x), np.log10(y)
    ax.scatter(x1, y1, c = 'steelblue', edgecolors = 'white', s = 90)
    ax.set_title('hHEP vs. {}'.format(col))
    ax.set_ylabel('hHEP\nProtein copy number [Log10]')
    ax.set_xlabel('Protein copy number [Log10]\n{}'.format(col))
    i += 1  
plt.savefig('figures/oxidative phosphorylation.pdf')

#### Protein copy number fraction of Kegg pathway

In [ ]:
data = data_cpno_median.copy()
cols = ['Liver', 'hHEP', 'hLSEC', 'hHSC', 'hKC']
data = data[cols]
data['Gene name'] = data.index.map(IDmapping_Perseus_UniprotAC_to_Genenames)
data = data.rename_axis('Samples', axis = 1)
data = data.set_index('Gene name')
kegg_frac = calculate_pathway_percentage(pathway_dictionary=pathway_dict, data = data)

In [ ]:
df = data_lfq.copy()
df['Gene name'] = df.index.map(IDmapping_Perseus_UniprotAC_to_Genenames)

In [ ]:
histones = [i for i in df.index if IDmapping_Perseus_UniprotAC_to_Genenames[i].startswith('HIST')]

In [ ]:
total_cpno = np.array(data.sum())

In [ ]:
data.sum()

In [ ]:
df_new = kegg_frac.T * total_cpno
kegg_frac = df_new.T

In [ ]:
df = kegg_frac.copy().T.sort_values(by = 'Liver', ascending = False)
df_top10 = df.iloc[:10, :].reset_index()
df_top10 = df_top10.melt(id_vars = 'Pathways')
metabolic_genes = list(set(pathway_dict['Metabolic pathways']) & set(data.index))

In [ ]:
df2 = data_cpno_log10.copy()
df2['Gene name'] = df2.index.map(IDmapping_Perseus_UniprotAC_to_Genenames)
df2 = df2.set_index('Gene name')
df2 = df2.rename_axis('Samples', axis = 1)
top10genes = data.loc[metabolic_genes].sort_values(by = 'hHEP', ascending = False).index[:10]
df2 = df2.rename(mapper = dict(zip(labelfile['Label4'], labelfile['Label3'])), axis = 1)
df2 = df2.loc[top10genes][cols].dropna()
df2_long = df2.T.reset_index().melt(id_vars = 'Samples')

In [ ]:
df = df_top10.copy()
fig, axes = plt.subplots(1, 2, figsize = (9, 4))
plt.subplots_adjust(wspace = 0.4)
palette = sns.diverging_palette(10, 220, sep=80, n=5)
sns.barplot(x='value', y='Pathways', hue='Samples', data=df, ax=axes[0], palette=palette, edgecolor='black')
sns.boxplot(x ='value', y='Gene name', hue='Samples', data=df2_long, ax=axes[1], palette=palette)
plt.savefig('figures/protein copy number kegg.pdf', bbox_inches='tight')

## Figure 6

#### Depth

In [ ]:
data = data_pcc_raw.copy().astype(np.number)
data.columns = data.columns.droplevel(0)
grouping_pcc = dict(zip(labelfile_pcc['Sample ID'], labelfile_pcc['Label2']))

In [ ]:
depth_pg = data.count().groupby(grouping_pcc).median()
error = data.count().groupby(grouping_pcc).std().tolist()
error.append(0)
depth_pg['total'] = data.dropna(how='all').shape[0]
fig, ax = plt.subplots(figsize = (4,4))
plt.bar(x = depth_pg.index, height = depth_pg, width = 0.4, edgecolor= 'black', facecolor = 'steelblue', yerr = error)
for i in np.arange(10):
    plt.annotate(depth_pg.iloc[i], [depth_pg.index[i], depth_pg.iloc[i]])
plt.ylabel('Protein groups')
plt.xticks(rotation = 20)
plt.title('Proteome depth');
plt.savefig('figures/PCC/proteome_depth.pdf', dpi=120, bbox_inches='tight')

#### Similarity

In [ ]:
data = data_pcc_log2.copy().astype(np.number)
data.columns = data.columns.droplevel(0)
data = data.rename(mapper = grouping_pcc, axis=1)

In [ ]:
df = data.corr()
cols = df.columns.unique().tolist()
combos = [i for i in it.combinations(cols,2)]
values = {}
for i in cols:
    values[i] = df.loc[i, i].replace(1, np.nan).mean().mean().round(2)
    
for i in combos: 
    values[i] = df.loc[i[0], i[1]].replace(1, np.nan).mean().mean().round(2)

In [ ]:
sns.clustermap(data = data.corr(), cmap='bwr', linewidth=.3, figsize=(4,4))
plt.savefig('figures/PCC/cluster.pdf', dpi=120, bbox_inches='tight')

#### PCA of all primary cell culture samples

In [ ]:
data = data_pcc_log2_impute.copy()
data.columns = data.columns.droplevel(0)

In [ ]:
df_PCA = data.rename(mapper = grouping_pcc, axis = 1)
df_PCA = df_PCA.rename_axis('Sample type', axis = 1)
df = df_PCA.T.reset_index()
features = df.columns[1:]

In [ ]:
x=df.loc[:, features].values
y=df.loc[:, 'Sample type'].values
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])
finalDf = pd.concat([principalDf, df['Sample type']], axis = 1)
PC1= pca.explained_variance_ratio_[0]
PC2= pca.explained_variance_ratio_[1]
PC1 = "{:.1%}".format(PC1)
PC2 = "{:.1%}".format(PC2)

In [ ]:
fig, ax = plt.subplots(figsize = (4,4))
ax.set_xlabel('Principal Component 1 ('+ str(PC1) + ')')
ax.set_ylabel('Principal Component 2 ('+ str(PC2) + ')')
ax.set_title('Principal component analysis')

targets = df['Sample type'].unique()
for target in targets:
    indicesToKeep = finalDf['Sample type'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , cmap =  'set3'
               , s = 80, edgecolors='black')
ax.legend(targets, bbox_to_anchor = [1.05, 0.8], framealpha=0.1)
ax.grid()
plt.savefig('figures/PCC/PCA_all.pdf', dpi=120, bbox_inches='tight')

#### Expression profiles

In [ ]:
data = data_pcc_log2_impute.copy()['hHSC']
data.columns = data.columns.map(mapper = grouping_pcc)

In [ ]:
df_long = data.reset_index().melt(id_vars='Protein ID', value_name='Intensity', var_name='Cell type').set_index('Protein ID')
df_long['Gene name'] = df_long.index.map(IDmapping_pcc_UniprotID_to_Genename)
df_long = df_long.reset_index().set_index('Gene name')

proteins_gf = ['IGFBP3', 'IGFBP4', 'IGFBP7', 'EGFR', 'FGF2', 'HGFAC', 'TGFBR1', 'TGFBR2', 'NRP1', 'NRP2', 'ENG']
proteins_cirr = ['COL1A1', 'COL2A1', 'COL3A1', 'LOXL1', 'LOXL2', 'MMP2', 'MMP14', 'TIMP2', 'DPP4', 'ANPEP', ]

df_gf = df_long.loc[proteins_gf].reset_index()
df_cirr = df_long.loc[proteins_cirr].reset_index()

In [ ]:
proteins_ecms=['ACTA2', 'COL1A1', 'COL2A1', 'COL3A1', 'LOXL2', 'LOXL4', 'MMP2', 'MMP14', 'TIMP2', 'DPP4', 'ANPEP']
proteins_gf = ['EGFR', 'FGF2', 'HGFAC', 'TGFBR1', 'TGFBR2', 'IGFBP3', 'IGFBP4', 'IGFBP7']
proteins_immune = ['IL6', 'C3', 'C4A', 'C5', 'C8B', 'VCAM1']
proteins_ald = ['VCAM1', 'IGFBP7', 'IGFALS', 'LGALS3BP']
proteins_all = proteins_ecms + proteins_gf + proteins_immune + proteins_ald
proteins_apoptosis=['CASP8', 'HGF']

In [ ]:
fold_changes = {}
for protein in proteins_all:
    d1 = df_long.loc[protein].groupby('Cell type').mean().loc['hHSC_D1']
    d7 = df_long.loc[protein].groupby('Cell type').mean().loc['hHSC_D7']
    fc = float(2**(d7-d1)-1)
    fc = round(fc, 2)
    fold_changes[protein]=fc

In [ ]:
sns.set_context("notebook", font_scale=1.4, rc={"lines.linewidth": 2})
sns.set_style('ticks')
for protein in proteins_apoptosis:
    fig, ax=plt.subplots(figsize=(1.4,1.4))
    sns.pointplot(data=df_long.loc[protein], x ='Cell type', y='Intensity', color='darkred', ci='sd').set_title(protein)
    plt.xticks(ticks=[0, 1, 2], labels=['D1', 'D3', 'D7'])
    plt.xlabel('')
    ax.set_ylabel('')
    #plt.savefig('figures/PCC/boxplot/HSC/{}.pdf'.format(protein), dpi=120, bbox_inches='tight')

#### GO enrichment plot

In [ ]:
df_hsc_up = pd.read_csv('datasets/PCC/GOresults/GO-Perseus/HSC-up-plot.txt', sep='\t').set_index('Name').sort_values(by='P value [-Log10]', ascending=False)
df_hep_down = pd.read_csv('datasets/PCC/GOresults/GO-Perseus/HEP-down_plot.txt', sep='\t').set_index('Name').sort_values(by='P value [-Log10]', ascending=False)

In [ ]:
sns.heatmap(df_hsc_up[['P value [-Log10]']], linewidth=0.6)
plt.savefig('figures/PCC/GO/hsc_up.pdf', dpi=120, bbox_inches='tight')

In [ ]:
sns.heatmap(df_hsc_up[['Enrichment']], linewidth=0.6)
plt.savefig('figures/PCC/GO/hsc_up2.pdf', dpi=120, bbox_inches='tight')

In [ ]:
sns.heatmap(df_hep_down[['P value [-Log10]']], linewidth=0.6)
plt.savefig('figures/PCC/GO/hep_down.pdf', dpi=120, bbox_inches='tight')

In [ ]:
sns.heatmap(df_hep_down[['Enrichment']], linewidth=0.6)
plt.savefig('figures/PCC/GO/hep_down2.pdf', dpi=120, bbox_inches='tight')

## Figure 7

In [ ]:
from src.functions import ancova_pg, pairwisetukey_pg, homoscedasticity_pg, normality_pg

In [ ]:
data=data_patients_long.copy().set_index('Protein ID').dropna()
data['gender']=data['gender'].replace({'m':1, 'f':0})
data=data.rename({'Intensity [Log2]':'Intensity'}, axis=1)

#### Homoscedasiticity test

In [ ]:
homoscedasticity_results = homoscedasticity_pg(data=data, dv='Intensity', group='grouping')

In [ ]:
homoscedasticity_results['equal_var'].value_counts(1)

#### Normality test

In [ ]:
normality_results = normality_pg(data=data, dv='Intensity', group='grouping')

In [ ]:
normality_results['normal'].value_counts(1)

#### ANCOVA

In [ ]:
ancova_results_bmi = ancova_pg(data=data, dv='Intensity', covar=['BMI'], between='grouping')
ancova_results_age_sex_bmi = ancova_pg(data=data, dv='Intensity', covar=['age', 'gender', 'BMI'], between='grouping')

In [ ]:
print('significant proteins in ancova after correcting for bmi: {}'.format(ancova_results_bmi[ancova_results_bmi.rejected].shape[0])) 
print('significant proteins in ancova after correcting for age, sex and bmi: {}'.format(ancova_results_age_sex_bmi[ancova_results_age_sex_bmi.rejected].shape[0])) 

In [ ]:
ancova_results = ancova_pg(data=data, dv='Intensity', covar=['age', 'gender'], between='grouping')

In [ ]:
sig_proteins_ancova=ancova_results[(ancova_results.rejected) & (ancova_results['Source']=='grouping')]
sig_proteins_ancova['Gene name']=sig_proteins_ancova['protein'].map(IDmapping_patients_UniprotID_to_GeneName)
sig_proteins_ancova=sig_proteins_ancova.rename({'protein':'Protein ID'}, axis=1)

In [ ]:
sig_proteins_ancova.to_csv('results/patients_ancova_sigproteins.csv')

#### Tukey_HSD

In [ ]:
data_tukey = data.loc[sig_proteins_ancova['Protein ID'].tolist()]
tukey_results = pairwisetukey_pg(data=data_tukey, dv='Intensity', between ='grouping')

In [ ]:
sig_proteins = sig_proteins_ancova

In [ ]:
df_heatmap = data_patients_long.groupby(['Protein ID', 'grouping']).mean()['Intensity [Log2]'].unstack().loc[sig_proteins_ancova['Protein ID']]
df_heatmap = df_heatmap.reindex(columns=['control', 'NASH', 'Cirrhosis'])

In [ ]:
df_heatmap_export = df_heatmap.assign(GeneName=df_heatmap.index.map(IDmapping_patients_UniprotID_to_GeneName))
df_heatmap_export.to_csv('datasets/Patients/PatientSamples_ANCOVA_sigproteins.csv')

#### Reviewer question: among proteins that are significantly upregulated in liver biopsies of patients with liver cirrhosis, are there any specific to HepA or PorV?

In [ ]:
#overlap with proteins specific to hepA or PortV?
overlap_ids = set(sig_proteins['Protein ID']) & set(unique_in_vessels_proteinids)
overlap_ids = set(sig_proteins['Protein ID']) & set(unique_in_eithervessel_proteinids)

[IDmapping_Perseus_UniprotAC_to_Genenames[i] for i in overlap_ids]

#### Abundance profile of CYP450 family members

In [ ]:
proteins_pts = ['PDGFRA', 'PDGFRB', 'TGFB1', 'TGFB1I1', 'TGFBI', 'LTBP1','LTBP2', 'LTBP4', 'ENG']
proteins_cyps_sig = ['CYP7B1', 'CYP11B1', 'CYP3A7', 'CYP3A4', 
                    'CYP4F3', 'CYP8B1', 'CYP1A2', 'CYP4A11', 
                    'CYP4F11', 'CYP4F2', 'CYP2C8', 'CYP27A1', 
                    'CYP2D6', 'CYP2C18', 'CYP5A1']

proteins_cyps_sig_ancova = set(sig_proteins['Gene name']) & set(proteins_cyps_sig)
proteins_pts_sig_ancova = set(sig_proteins['Gene name']) & set(proteins_pts)

In [ ]:
data = data_patients_long.set_index('Gene name')
df_cyp=data.loc[proteins_cyps_sig_ancova].reset_index().groupby(['Gene name', 'grouping']).mean().unstack('grouping')['Intensity [Log2]']
df_cyp=df_cyp.reindex(columns=['control', 'NASH', 'Cirrhosis'])

In [ ]:
sns.clustermap(data=df_cyp, z_score=0, cmap='bwr', col_cluster=False, 
              linecolor='white', linewidth=2, figsize=(3,5), yticklabels=True, xticklabels=False)
plt.savefig('figures/Patients/heatmap/cyp450.pdf', dpi=120, bbox_inches='tight')

#### Abundance profile of growth factors

In [ ]:
sns.set_context("notebook", font_scale=1.4, rc={"lines.linewidth": 2})
sns.set_style('ticks')
for protein in proteins_pts:
    fig, ax=plt.subplots(figsize=(1.4,1.4))
    sns.boxplot(data=data.loc[protein], x ='grouping', y='Intensity [Log2]', 
                width=0.6, order=['control', 'NASH', 'Cirrhosis'], color='white', linewidth=1).set_title(protein)
    plt.xticks(ticks=[0, 1, 2], labels=[''], rotation=30)
    plt.xlabel('')
    plt.ylabel('')
    ymin, ymax = plt.ylim()
    plt.ylim(ymin*0.9, ymax*1.05)
    for _,s in ax.spines.items():
        s.set_linewidth(1)
        s.set_color('black')
    for i,box in enumerate(ax.artists):
        box.set_edgecolor('black')
        box.set_facecolor('white')

    # iterate over whiskers and median lines
        for j in range(6*i,6*(i+1)):
             ax.lines[j].set_color('black')
    plt.savefig('figures/Patients/boxplot/{}.pdf'.format(protein), dpi=120, bbox_inches='tight')

#### Significantly enriched GO terms

In [ ]:
patient_go = pd.read_csv('datasets/Patients/Patients_GO_toplot_ANCOVA.csv').set_index('Name')
patient_go_up = patient_go[patient_go['regulation']=='up'].sort_values(by='FDR [-Log 10]', ascending=False)
patient_go_down = patient_go[patient_go['regulation']=='down'].sort_values(by='FDR [-Log 10]', ascending=False)

In [ ]:
for i in patient_go_up.index:
    print(i)

In [ ]:
sns.heatmap(patient_go_up[['Fold Enrichment']], linewidth=0.6)
plt.savefig('figures/Patients/go/up_fc.pdf', dpi=120, bbox_inches='tight')

In [ ]:
sns.heatmap(patient_go_up[['FDR [-Log 10]']], linewidth=0.6)
plt.savefig('figures/Patients/go/up_pvalue.pdf', dpi=120, bbox_inches='tight')

In [ ]:
sns.heatmap(patient_go_down[['Fold Enrichment']], linewidth=0.6)
plt.savefig('figures/Patients/go/down_fc.pdf', dpi=120, bbox_inches='tight')

In [ ]:
sns.heatmap(patient_go_down[['FDR [-Log 10]']], linewidth=0.6)
plt.savefig('figures/Patients/go/down_pvalue.pdf', dpi=120, bbox_inches='tight')

## Figure EV3

#### Protein copy number vs. molecular weight

In [ ]:
data = data_cpno_median.copy()
data['Mol. weight [kDa]'] = data.index.map(dict(zip(MW['Protein ID'], MW['Mol. weight [kDa]'])))
data = data.dropna(subset = ['hHEP'])
x = np.log10(data['Mol. weight [kDa]'])
y = np.log10(data['hHEP'])

plt.figure(1, figsize=(4, 4))
axScatter = plt.axes([0.1, 0.1, 0.65, 0.65])
axHistx = plt.axes([0.8, 0.1, 0.12, 0.65])
axHisty = plt.axes([0.1, 0.8, 0.65, 0.12])
axScatter.scatter(x, y, c='steelblue', edgecolors='white')
axScatter.set_ylabel('Copy number [log10]')
axScatter.set_xlabel('Mol. weight [Da] [log10]')
axScatter.set_xlim(3, 7)
axScatter.set_ylim(2, 9)
axHisty.hist(x, color='steelblue', alpha = 0.5, edgecolor = 'black', bins=40, label='Mol. weight [Da]')
axHisty.set_xlim(3,7)
axHisty.set_xticks(range(3,8))
axHistx.hist(y, color='steelblue', alpha = 0.5, bins=40, range=(2, 9),edgecolor = 'black', orientation='horizontal', label='Copy number')
axHistx.set_ylim(2,9);

#### Estimated total cellular protein content

In [ ]:
total_protein_percell = pd.read_csv(Copynumber_overview, sep='\t')[1:]
col_to_keep = ['Total protein [pg/cell]', 'Histone mass [pg/cell]', 'Total molecules per cell', 'Ploidy', 'Sample']
total_protein = total_protein_percell[col_to_keep].merge(labelfile[['Sample', 'Label3']], on = 'Sample')
for i in col_to_keep[:3]:
    total_protein[i] = total_protein[i].astype(np.number)
stats = total_protein.groupby(['Label3']).describe()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (8, 4))
plt.subplots_adjust(wspace = 0.3)
col = 'Total protein [pg/cell]'
x = stats.index
y = stats[col]['mean']
error = stats[col]['std']
ax1.bar(x = x, height = y, yerr = error, edgecolor= 'white', facecolor = 'steelblue', 
        error_kw=dict(ecolor='gray', capsize=4))
ax1.set_xticklabels(labels = x, rotation = 45)
ax1.set_ylabel('Total protein per cell (pg)')

col = 'Total molecules per cell'
y = stats[col]['mean']
error = stats[col]['std']
ax2.bar(x = x, height = y, yerr = error, edgecolor = 'white', facecolor = 'steelblue', 
       error_kw=dict(ecolor='gray', capsize=4))
ax2.set_xticklabels(labels = x, rotation = 45)
ax2.set_ylabel('Total protein copy number per cell (1x 1e10)');

#### Proteins of similar abundance profile to uncharacterized protein H0YL77

In [ ]:
df_test = data_lfq.T.dropna(subset=['H0YL77'])
df_test1 = df_test.T.dropna().T
corr_to_h0yl77 = pg.pairwise_corr(data=df_test1, columns=['H0YL77']).sort_values(by='r', ascending = False)

np.log10(df_test[['Q9GZY8-2', 'H0YL77']]).plot()
plt.title('Uncharacterized protein in hHEP')
plt.ylabel('LFQ intensity [Log10]')

In [ ]:
corr_to_h0yl77.to_csv('results/correlation_to_h0yl77.csv')

#### Protein copy number in SNPs and drug targets

In [ ]:
df=data_cpno_median.copy()
df['Gene name']=df.index.map(IDmapping_Perseus_UniprotAC_to_Genenames)
targets=pd.read_csv('annotations/copynumber_gf_snps.csv')['gf and snps'].tolist()
AA = df[df['Gene name'].isin(targets)][['Liver', 'hHEP', 'hLSEC', 'hKC', 'hHSC', 'Gene name']].dropna(subset=['hHSC'])
AA.to_csv('results/copynumber_snps_drugtargets.csv')

## Figure EV1

In [ ]:
data_para = extract_Parameters(ProteinGroups)
proteins_use = data_lfq.index
data_para = data_para.loc[proteins_use]

In [ ]:
df = data_para.copy()
SequenceCoverage_Cols = [col for col in df if col.startswith('Sequence coverage')]
Uniquepeptide_Cols = [col for col in df if col.startswith('Unique peptides')]
Peptide_Cols = [col for col in df if col.startswith('Peptides')]
df_seqcov = df[SequenceCoverage_Cols]
df_uniquepeptides = df[Uniquepeptide_Cols]
df_peptides=df[Peptide_Cols]

cols = list(df_seqcov.columns)
new_cols = [col.split(' ')[2] for col in cols]
new_cols[0] = 'total'
for df in [df_seqcov, df_uniquepeptides, df_peptides]:
    df.columns = new_cols

In [ ]:
grouping2 = dict(zip(labelfile.Sample, labelfile.Label3))
grouping2['total'] = 'total'
df_seqcov_median = df_seqcov.groupby(grouping2, axis = 1).median()
df_peptides_median = df_peptides.groupby(grouping2, axis = 1).median()
df_uniquepeptides_median = df_uniquepeptides.groupby(grouping2, axis=1).median()

In [ ]:
df_seqcov_median_long = df_seqcov_median.reset_index().melt(id_vars=['Protein ID'],value_name='sequence coverage', var_name='sample type')
df_peptides_median_long = df_peptides_median.reset_index().melt(id_vars=['Protein ID'], value_name='Peptides per protein', var_name='sample type')

#### Sequence coverage across sample types

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
sns.boxplot(x='sample type', y ='sequence coverage', data=df_seqcov_median_long, 
            color='lightblue', width=0.6, linewidth=1)
plt.xticks(rotation=30)
plt.yticks(fontsize=15)
plt.ylim(-5, 105)
plt.ylabel('Sequence coverage [%]', fontsize=15)
plt.title('Sequence coverage', fontsize=20)
plt.savefig('figures/FigureEV1_a.pdf', bbox_inches='tight', dpi=120)

In [ ]:
fig, ax=plt.subplots(figsize=(2,6))
plt.hist(df_seqcov_median['total'], orientation='horizontal', color='lightblue', edgecolor='black', range=(-5, 105), bins=30)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title('Distribution', fontsize=20)
plt.savefig('figures/FigureEV1_b.pdf', bbox_inches='tight', dpi=120)

#### Identified peptides per protein group across sample types

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
sns.boxplot(x='sample type', y ='Peptides per protein', data=df_peptides_median_long, 
            color='lightblue', width=0.6, linewidth=1)
plt.xticks(rotation=30)
plt.yticks(fontsize=15)
plt.ylabel('Peptides per PG', fontsize=15)
plt.title('Peptides per protein group', fontsize=20)
plt.savefig('figures/FigureEV1_c.pdf', bbox_inches='tight', dpi=120)

In [ ]:
fig, ax=plt.subplots(figsize=(2,6))
plt.hist(df_peptides_median['total'], orientation='horizontal', color='lightblue', 
         edgecolor='black', range=(-5, 105), bins=50)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title('Distribution', fontsize=20)
plt.savefig('figures/FigureEV1_d.pdf', bbox_inches='tight', dpi=120)

#### Dataset summary for the webpage

In [ ]:
data_summary = pd.DataFrame(index = data_lfq.index)
data_summary['Gene name'] = data_summary.index.map(IDmapping_Perseus_UniprotAC_to_Genenames)
data_summary['Protein name'] = data_summary.index.map(IDmapping_Perseus_UniprotAC_to_Proteinnames)
data_summary['Genename_ProteinID'] = data_summary.index.map(IDmapping_Perseus_UniprotAC_to_Genename_UniID)
data_summary['Mol. weight [Da]'] = data_summary.index.map(dict(zip(MW['Protein ID'], MW['Mol. weight [kDa]'])))
data_summary['Sequence coverage [%]'] = df_seqcov_median['total']
data_summary['Peptides per protein'] = df_peptides_median['total']
data_summary['Unique peptides per protein'] = df_uniquepeptides['total']

In [ ]:
data_summary.reset_index().to_csv('datasets/Atlas/data_summary.csv', index= False)

#### Peptides information for the webpage

In [ ]:
peptide = pd.read_csv('datasets/Atlas/peptides.txt', sep = '\t', low_memory = False)
peptide = peptide[peptide['Reverse']!='+']

In [ ]:
use_cols = ['Sequence', 'Length', 'Mass', 'Proteins', 'Leading razor protein', 'Unique (Groups)', 'Unique (Proteins)', 'Score', 'MS/MS Count', 'Intensity']
data_peptide = peptide[use_cols]
data_peptide = data_peptide.set_index('Leading razor protein')
data_peptide['Genename_ProteinID'] = data_peptide.index.map(IDmapping_Perseus_UniprotAC_to_Genename_UniID)
data_peptide = data_peptide.reset_index()
new_order = ['Sequence', 'Length', 'Mass', 'Score', 'Intensity', 'MS/MS Count', 'Genename_ProteinID', 'Proteins',  'Unique (Groups)', 'Unique (Proteins)']
data_peptide = data_peptide[new_order]
data_peptide['Score'] = data_peptide['Score'].round()
data_peptide.to_csv('datasets/data_peptide.csv', index = False)